In [9]:
import pandas as pd
import numpy as np
import math
import time

In [10]:
train=pd.read_csv("train.csv")
evaluation=pd.read_csv("evaluation.csv")

In [11]:
#extract week day and hour from the timestamp

def preprocess_day(timestamp):
    dico_day={"Mon":0,"Tue":1,"Wed":2,"Thu":3,"Fri":4,"Sat":5,"Sun":6}
    res=7*[0]
    res[dico_day[time.ctime(int(str(timestamp)[:-3]))[:3]]]=1
    return res

def preprocess_time(timestamp):
    res=24*[0]
    res[int(time.ctime(int(str(timestamp)[:-3]))[11:13])]=1
    return res

df=pd.DataFrame(train["timestamp"])
train["day"] = df.applymap(lambda x: preprocess_day(x))
train["time"] = df.applymap(lambda x: preprocess_time(x))
train = train.drop(columns="timestamp")

df=pd.DataFrame(evaluation["timestamp"])
evaluation["day"] = df.applymap(lambda x: preprocess_day(x))
evaluation["time"] = df.applymap(lambda x: preprocess_time(x))
evaluation = evaluation.drop(columns="timestamp")

In [12]:
#replace the boolean by its int value
df=pd.DataFrame(train["user_verified"])
train["user_verified"]=df.applymap(lambda x: int(x))

df=pd.DataFrame(evaluation["user_verified"])
evaluation["user_verified"]=df.applymap(lambda x: int(x))

In [13]:
#centrer & réduire --> on utilise les paramètres stats des données d'entraînement
cols=["user_statuses_count","user_followers_count","user_friends_count"]
for c in cols:
    df=pd.DataFrame(train[c])
    df2=pd.DataFrame(evaluation[c])
    m=train[c].mean()
    v=math.sqrt(train[c].var())
    train[c]=df.applymap(lambda x: (x-m)/v)
    evaluation[c]=df2.applymap(lambda x: (x-m)/v)

In [14]:
#count the number of 
def count_occ(s):
    if type(s)==str:
        return len(s.split(","))
    else:
        return 0

cols=["user_mentions","urls","hashtags"]
for c in cols:
    df=pd.DataFrame(train[c])
    df2=pd.DataFrame(evaluation[c])
    train[c]=df.applymap(lambda x: count_occ(x))
    evaluation[c]=df2.applymap(lambda x: count_occ(x))

In [18]:
evaluation=evaluation.drop(columns=["text"])

In [15]:
np.save("data_preprocessed.npy",data)
np.save("data_columns.npy",list(data.columns))

In [19]:
np.save("test_preprocessed.npy",evaluation)
np.save("test_columns.npy",list(evaluation.columns))